In [ ]:
import os,glob
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import numpy as np

dataInputPath='data/volumes'
imagePathInput=os.path.join(dataInputPath,'img/')
maskPathInput=os.path.join(dataInputPath,'mask/')

dataOutputPath='data/slices'
imageSliceOutput=os.path.join(dataOutputPath,'img/')
maskSliceOutput=os.path.join(dataOutputPath,'mask')
imgPath=os.path.join(imagePathInput,'volume-0.nii')
img=nib.load(imgPath).get_fdata()
np.min(img),np.max(img),img.shape,type(img)


maskPath=os.path.join(maskPathInput,'segmentation-0.nii')
mask=nib.load(maskPath).get_fdata()
np.min(mask),np.max(img),mask.shape,type(mask)
imgSlice=mask[:,:,65]
plt.imshow(imgSlice,cmap='gray')
plt.show()

HOUNSFIELD_MIN=-1000 ####### may be i will have to change these valuse accoriding to values of my images
HOUNSFIELD_MAX=2000
HOUNSFIELD_RANGE=HOUNSFIELD_MAX-HOUNSFIELD_MIN
def normalizeImageIntensityRange(img):
    img[img<HOUNSFIELD_MIN]=HOUNSFIELD_MIN
    img[img>HOUNSFIELD_MAX]=HOUNSFIELD_MAX
    return (img-HOUNSFIELD_MIN)/HOUNSFIELD_RANGE

##this is used to normalze the images
def readImageVolume(imgPath,normalize=False):
    img=nib.load(imgPath).get_fdata()
    if normalize:
        return normalizeImageIntensityRange(img)
    else:
        return img

#this is used to read the images

def saveSlice(img,fname,path):
    img=np.uint8(img*255)
    fout=os.path.join(path,f'{fname}.png')
    cv2.imwrite(fout,img)
    print(f'[+] Slice saved:{fout}',end='\r')
SLICE_DECIMATE_IDENTIFIER=3
SLICE_X=True
SLICE_Y=True
SLICE_Z=True
def sliceAndSaveVolumeImage(vol,fname,path):
    (dimx,dimy,dimz)=vol.shape
    count=0
    if SLICE_X:
        count+=dimx
        print('Slicing X:')
        for i in range(dimx):
            saveSlice(vol[i,:,:],fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x',path)

    
    if SLICE_Y:
        count+=dimy
        print('Slicing Y:')
        for i in range(dimy):
            saveSlice(vol[:,i,:],fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y',path)
    
    
    if SLICE_Z:
        count+=dimz
        print('Slicing Z:')
        for i in range(dimz):
            saveSlice(vol[:,:,i],fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z',path)
    
for index,filename in enumerate(sorted(glob.iglob(imagePathInput+'*.nii'))):
    print('index of the file being sliced : ',index)
    img=readImageVolume(filename,True)
    print(filename,img.shape,np.sum(img.shape),np.min(img),np.max(img))
    numofSlices=sliceAndSaveVolumeImage(img,'volume-'+str(index),imageSliceOutput)
    print(f'\n{filename},{numofSlices} slices created \n')
for index,filename in enumerate(sorted(glob.iglob(maskPathInput+'*.nii'))):
    print('index of the file being sliced : ',index)
    img=readImageVolume(filename,True)
    print(filename,img.shape,np.sum(img.shape),np.min(img),np.max(img))
    numofSlices=sliceAndSaveVolumeImage(img,'volume-'+str(index),maskSliceOutput)
    print(f'\n{filename},{numofSlices} slices created \n')
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

SEED=909
BATCH_SIZE_TRAIN=32
BATCH_SIZE_TEST=32

IMAGE_HEIGHT=40
IMAGE_WIDTH=80
IMG_SIZE=(IMAGE_HEIGHT,IMAGE_WIDTH)

data_dir='data/slices/'
data_dir_train=os.path.join(data_dir,'training')
data_dir_train_image=os.path.join(data_dir_train,'img')
data_dir_train_mask=os.path.join(data_dir_train,'mask')

data_dir_test=os.path.join(data_dir,'test')
data_dir_test_image=os.path.join(data_dir_test,'img')
data_dir_test_mask=os.path.join(data_dir_test,'mask')


def unet(n_levels,initial_features=32,n_blocks=2,kernel_size=3,pooling_size=2,in_channels=1,out_channels=1):
    inputs=keras.layers.Input(shape=(IMAGE_HEIGHT,IMAGE_WIDTH,in_channels))
    x=inputs

    convpars=dict(kernel_size=kernel_size,activation='relu',padding='same')

    #downstream
    skips={}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x=keras.layers.Conv2D(initial_features*2**level,**convpars)(x)
        if level<n_levels-1:
            skips[level]=x
            x=keras.layers.MaxPool2D(pooling_size)(x)

    #upstream
    for level in reversed(range(n_levels-1)):
        x=keras.layers.Conv2DTranspose(initial_features*2**level,strides=pooling_size,**convpars)(x)
        x=keras.layers.Concatenate()([x,skips[level]])
        for _ in range(n_blocks):
            x=keras.layers.Conv2D(initial_features*2**level,**convpars)(x)

    


    #output
    x=keras.layers.Conv2D(out_channels,kernel_size=1,activation='sigmoid',padding='same')(x)

    return keras.Model(inputs=[inputs],outputs=[x],name=f'UNET_L{n_levels}-F{initial_features}')

NUM_TRAIN=0
NUM_TEST=0

NUM_OF_EPOCHS=0


EPOCHS_STEP_TRAIN=NUM_TRAIN//BATCH_SIZE_TRAIN
EPOCHS_STEP_TEST=NUM_TEST//BATCH_SIZE_TEST

model=unet(4)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])





model.summary()
model.fit_generator(generator=train_generator,
                    steps_per_epoch=EPOCHS_STEP_TRAIN,
                    validation_steps=EPOCHS_STEP_TEST,
                    epochs=NUM_OF_EPOCHS)
